# Database initialization

Convert the csv files from kaggle to postgresql database.

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from psycopg2 import connect, extensions, sql
import csv
import sys
import traceback
from IPython.display import clear_output
from io import StringIO
from datetime import datetime, timedelta
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile

In [2]:
api = KaggleApi()
api.authenticate()

In [3]:
# delete the archive folder if it already exists
if os.path.exists('archive'): 
    shutil.rmtree('archive')

# download the F1 dataset from kaggle
api.dataset_download_files("rohanrao/formula-1-world-championship-1950-2020")
zf = ZipFile('formula-1-world-championship-1950-2020.zip')
zf.extractall('archive\\')
zf.close()

In [4]:
# get login info

with open("postgres_login_info.txt") as f: 
    login_info = f.read()

In [5]:
# Circuits table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # delete old table
    cur.execute("DROP TABLE IF EXISTS circuits CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE circuits(
     circuitId  INTEGER NOT NULL PRIMARY KEY, 
     circuitRef VARCHAR(100) NOT NULL, 
     name       VARCHAR(250) NOT NULL, 
     location   VARCHAR(100) NOT NULL, 
     country    VARCHAR(100) NOT NULL, 
     lat        DECIMAL NOT NULL, 
     lng        DECIMAL NOT NULL, 
     alt        INTEGER, 
     url        VARCHAR(250)
     )
     """
    cur.execute(create_table)
    
    # insert values 
    with open('archive/circuits.csv') as f:
        copy = sql.SQL("""
        COPY circuits FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [6]:
# constructors table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS constructors CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE constructors(
     constructorId  INTEGER NOT NULL PRIMARY KEY, 
     constructorRef VARCHAR(100) NOT NULL, 
     name           VARCHAR(100) NOT NULL, 
     nationality    VARCHAR(100), 
     url            VARCHAR(100)
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/constructors.csv") as f: 
        copy = sql.SQL("""
        COPY constructors FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [7]:
# drivers table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS drivers CASCADE")

    # create table
    create_table = """
     CREATE TABLE drivers( 
     driverId    INTEGER NOT NULL PRIMARY KEY, 
     driverRef   VARCHAR(100) NOT NULL, 
     number      INTEGER, 
     code        VARCHAR(3),  
     forename    VARCHAR(100) NOT NULL, 
     surname     VARCHAR(100) NOT NULL, 
     dob         DATE NOT NULL, 
     nationality VARCHAR(100) NOT NULL, 
     url         VARCHAR(100)
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/drivers.csv", encoding="utf-8") as f: 
        copy = sql.SQL("""
        COPY drivers FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [8]:
# seasons table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS seasons CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE seasons(
     year INTEGER NOT NULL PRIMARY KEY, 
     url  VARCHAR(150)
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/seasons.csv") as f: 
        copy = sql.SQL("""
        COPY seasons FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [9]:
# status table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS status CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE status(
     statusId INTEGER NOT NULL PRIMARY KEY, 
     status  VARCHAR(150)
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/status.csv") as f: 
        copy = sql.SQL("""
        COPY status FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [10]:
# constructor results table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # delete old table
    cur.execute("DROP TABLE IF EXISTS constructor_results CASCADE")
    
    # create table -- we need to use decimals for points becuase in the early days of F1 it was not uncommon for drivers
    # to recieve fractional points
    create_table = """
     CREATE TABLE constructor_results(
     constructorResultsId INTEGER NOT NULL PRIMARY KEY, 
     raceId               INTEGER NOT NULL REFERENCES races(raceId), 
     constructorId        INTEGER NOT NULL REFERENCES constructors(constructorId), 
     points               DECIMAL NOT NULL,  
     status               CHAR
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open('archive/constructor_results.csv') as f: 
        copy = sql.SQL("""
        COPY constructor_results FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [11]:
# constructor standings table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS constructor_standings CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE constructor_standings(
     constructorStandingsId INTEGER NOT NULL PRIMARY KEY, 
     raceId                 INTEGER NOT NULL REFERENCES races(raceId), 
     constructorId          INTEGER NOT NULL REFERENCES constructors(constructorId), 
     points                 DECIMAL NOT NULL, 
     position               INTEGER NOT NULL, 
     positionText           VARCHAR(10) NOT NULL, 
     wins                   INTEGER NOT NULL
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/constructor_standings.csv") as f: 
        copy = sql.SQL("""
        COPY constructor_standings FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [12]:
# driver standings table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS driver_standings CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE driver_standings(
     driverStandingsId INTEGER NOT NULL PRIMARY KEY, 
     raceId            INTEGER NOT NULL REFERENCES races(raceId), 
     driverId          INTEGER NOT NULL REFERENCES drivers(driverId), 
     points            DECIMAL NOT NULL, 
     position          INTEGER NOT NULL, 
     positionText      VARCHAR(3) NOT NULL, 
     wins              INTEGER NOT NULL
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/driver_standings.csv") as f: 
        copy = sql.SQL("""
        COPY driver_standings FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [13]:
# lap times

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS lap_times CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE lap_times(
     raceId       INTEGER NOT NULL REFERENCES races(raceId), 
     driverId     INTEGER NOT NULL REFERENCES drivers(driverId), 
     lap          INTEGER NOT NULL, 
     position     INTEGER NOT NULL,
     time         TIME NOT NULL, 
     milliseconds INTEGER NOT NULL
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/lap_times.csv") as f: 
        copy = sql.SQL("""
        COPY lap_times FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [14]:
# pit_stops table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS pit_stops CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE pit_stops(
     raceId       INTEGER NOT NULL REFERENCES races(raceId), 
     driverId     INTEGER NOT NULL REFERENCES drivers(driverId), 
     stop         INTEGER NOT NULL, 
     lap          INTEGER NOT NULL, 
     time         INTERVAL NOT NULL, 
     duration     INTERVAL, 
     milliseconds INTEGER NOT NULL
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/pit_stops.csv") as f: 
        reader = csv.reader(f)
        next(reader)
        for row in reader: 
            if ':' in row[5]:
                duration = datetime.strptime(row[5], "%M:%S.%f")
            else: 
                duration = timedelta(seconds=float(row[5]))
            
            insert = f"""
            INSERT INTO pit_stops VALUES(
            {int(row[0])}, 
            {int(row[1])}, 
            {int(row[2])}, 
            {int(row[3])}, 
            '{row[4]}', 
            '{row[5]}', 
            {int(row[6])} 
            )
            """
            
            cur.execute(insert)
        

In [15]:
# Qualifying table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS qualifying CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE qualifying(
     qualifyId INTEGER NOT NULL PRIMARY KEY, 
     raceId    INTEGER NOT NULL REFERENCES races(raceId), 
     driverId  INTEGER NOT NULL REFERENCES drivers(driverId), 
     constructorId INTEGER NOT NULL REFERENCES constructors(constructorId), 
     number INTEGER, 
     position INTEGER NOT NULL, 
     q1 TIME, 
     q2 TIME, 
     q3 TIME
     );
     """
    cur.execute(create_table)
    
    # replace NaN with '\\N' to match the rest of the dataset
    # TODO: during data cleaning, evaluate whether it might be more sensible to replace '\\N' with 0 to differentiate 
    # between no q1, q2 appearence becuase they were knocked out or becuase F1 was using a different quali format
    
    quali = pd.read_csv('archive/qualifying.csv')
    quali[['q1', 'q2', 'q3']] = quali[['q1', 'q2', 'q3']].fillna(value='\\N')
    quali.to_csv('archive/qualifying.csv', index=False)
    
    # insert values
    with open("archive/qualifying.csv") as f: 
        copy = sql.SQL("""
        COPY qualifying FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [16]:
# races table

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS races CASCADE")
    
    # create table
    create_table = """
     CREATE TABLE races(
     raceId INTEGER NOT NULL PRIMARY KEY, 
     year    INTEGER NOT NULL REFERENCES seasons(year), 
     round  INTEGER NOT NULL, 
     circuitId INTEGER NOT NULL REFERENCES circuits(circuitId), 
     name VARCHAR(100) NOT NULL, 
     date DATE NOT NULL, 
     time TIME, 
     url VARCHAR(150) NOT NULL
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/races.csv") as f: 
        copy = sql.SQL("""
        COPY races FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)

In [17]:
# results table
## TODO: figure out what's going on with 'time' during cleaning

with connect(login_info) as conn, conn.cursor() as cur:  
    autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
    conn.set_isolation_level(autocommit)
    
    # drop old table
    cur.execute("DROP TABLE IF EXISTS results")
    
    # create table
    create_table = """
     CREATE TABLE results(
     resultId        INTEGER NOT NULL PRIMARY KEY, 
     raceId          INTEGER NOT NULL REFERENCES races(raceId), 
     driverId        INTEGER NOT NULL REFERENCES drivers(driverId), 
     constructorId   INTEGER NOT NULL REFERENCES constructors(constructorId), 
     number          INTEGER, 
     grid            INTEGER, 
     position        INTEGER, 
     positionText    VARCHAR(10), 
     positionOrder   INTEGER, 
     points          DECIMAL, 
     laps            INTEGER, 
     time            VARCHAR(50), 
     milliseconds    INTEGER, 
     fastestLap      INTEGER, 
     rank            INTEGER, 
     fastestlapTime  TIME, 
     fastestLapSpeed DECIMAL, 
     statusId        INTEGER NOT NULL REFERENCES status(statusId)
     );
     """
    cur.execute(create_table)
    
    # insert values
    with open("archive/results.csv") as f: 
        copy = sql.SQL("""
        COPY results FROM STDIN WITH CSV HEADER NULL '\\N' """)

        cur.copy_expert(copy, f)